<h1><center> Text mining part </center></h1>

In [3]:
import requests
from bs4 import BeautifulSoup

In [5]:
add_page = requests.get('https://auto.bazos.cz/inzerat/152169281/volkswagen-golf-plus-19-tdibez-dpf-filtru77-kw.php')

In [6]:
soup_add = BeautifulSoup(add_page.text, 'html')

In [15]:
add = soup_add.find('div', {'class':'popisdetail'}).get_text()
print(add)

Volkswagen Golf Plus 1.9 TDi,výkon 77 Kw-NÍZKA SPOTŘEBA.Originál z výroby bez DPF filtru.Stav poctivě najetých kilometrů je 202160 Km-pravost kilometrů doložím.Lak vozidla je bez kosmetických vad,zachovalý NEKUŘÁCKÝ interiér viz foto-více foto zašlu na email.Rok 2006-při přihlášení vozidla se platí pouze 800 Kč.BYLA PROVEDENA KONTROLA V AUTOSERVISU-podvozkově a mechanicky vše v pořádku.Zadní sedačky s odkládacími stolečky a ISOFIXY.Bez koroze.16 ALU kola,zachovalé pneu (vzorek cca 70 %).STK platná do 10/2023.
Výbava:TAŽNÉ ZAŘÍZENÍ,ZADNÍ INDIKÁTOR PARKOVÁNÍ,TEMPOMAT,Dvouzonová digitální klimatizace,ABS, ESP, Protiprokluzový systém kol (ASR),
Centrální zamykání, Dálkové centrální zamykání,El. ovládání oken, El. ovládání zrcátek, Posilovač řízení,Autorádio, Palubní počítač,


In [14]:
print(soup_add.find('div', {'class':'popisdetail'}).get_text())

Volkswagen Golf Plus 1.9 TDi,výkon 77 Kw-NÍZKA SPOTŘEBA.Originál z výroby bez DPF filtru.Stav poctivě najetých kilometrů je 202160 Km-pravost kilometrů doložím.Lak vozidla je bez kosmetických vad,zachovalý NEKUŘÁCKÝ interiér viz foto-více foto zašlu na email.Rok 2006-při přihlášení vozidla se platí pouze 800 Kč.BYLA PROVEDENA KONTROLA V AUTOSERVISU-podvozkově a mechanicky vše v pořádku.Zadní sedačky s odkládacími stolečky a ISOFIXY.Bez koroze.16 ALU kola,zachovalé pneu (vzorek cca 70 %).STK platná do 10/2023.
Výbava:TAŽNÉ ZAŘÍZENÍ,ZADNÍ INDIKÁTOR PARKOVÁNÍ,TEMPOMAT,Dvouzonová digitální klimatizace,ABS, ESP, Protiprokluzový systém kol (ASR),
Centrální zamykání, Dálkové centrální zamykání,El. ovládání oken, El. ovládání zrcátek, Posilovač řízení,Autorádio, Palubní počítač,


In [8]:
def get_numbers_from_text(text):
    import re
    pattern = '[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?'
    list_of_numbers = re.findall(pattern, text)
    return list_of_numbers

In [18]:
all_numbers = get_numbers_from_text(add)
print(all_numbers)

['1.9', '77', '202160', '2006', '800', '.16', '70', '10', '2023.']


In [45]:
def get_context(text, list_of_tokens, context_span=20):
    import re
    context = []
    for token in list_of_tokens:
        all_occurences_indices = [m.start() for m in re.finditer(token, text)]
        for index in all_occurences_indices:
            left_index = max(index - context_span, 0)
            right_index = min(index + context_span, len(text))
            substring = text[left_index: right_index].strip()
            
            if left_index == 0:
                substring = ' '.join(substring.split(' ')[:-1])
            elif right_index == len(text):
                substring = ' '.join(substring.split(' ')[1:])
            else:
                substring = ' '.join(substring.split(' ')[1:-1])
            context.append(substring)
    return context
context_got = get_context(add, all_numbers)
print(context_got)

['Golf Plus 1.9 TDi,výkon 77', '1.9 TDi,výkon 77 Kw-NÍZKA', 'kilometrů je 202160 Km-pravost', 'na email.Rok 2006-při', 'se platí pouze 800 Kč.BYLA', 'kilometrů je 202160 Km-pravost', 'ISOFIXY.Bez koroze.16 ALU', 'pneu (vzorek cca 70 %).STK platná', '%).STK platná do', 'platná do']


In [51]:
def find_km(contexts):
    import re
    for text in contexts:
        test = [m.start() for m in re.finditer('Km', text)]
        if len(test) > 0:
            km_number = get_numbers_from_text(text)
    return km_number    
find_km(context_got)

['202160']

In [43]:
def get_context(text, list_of_tokens, context_span=20):
    import re
    context = []
    all_indexes = []
    for token in list_of_tokens:
        all_occurences_indices = [m.start() for m in re.finditer(token, text)]
        all_occurences_indices_ends = [m.end() for m in re.finditer(token, text)]
        
        if len(all_occurences_indices) > 1:
            all_occurences_indices = [max(all_occurences_indices)] #have to solve more sophistically
        if len(all_occurences_indices_ends) > 1:
            all_occurences_indices_ends = [max(all_occurences_indices_ends)]
        all_indexes.append(all_occurences_indices_ends)
        print(max(all_indexes))
        print(all_indexes[-1])
        
        for index in all_occurences_indices:
            print(index)
            left_index = max(min(index - context_span, index - all_indexes[-1]), 0)
            #left_index = max(index - context_span, 0)
            right_index = min(index + context_span, len(text))
            substring = text[left_index: right_index].strip()
            
            if left_index == 0:
                substring = ' '.join(substring.split(' ')[:-1])
            elif right_index == len(text):
                substring = ' '.join(substring.split(' ')[1:])
            else:
                substring = ' '.join(substring.split(' ')[1:-1])
            context.append(substring)
    return context, all_indexes
get_context(add, all_numbers)

[24]
[24]
21


TypeError: unsupported operand type(s) for -: 'int' and 'list'